# Function calling


In [1]:
from typing import Any
from openai.types.chat import ChatCompletionToolParam
from openai.types.shared_params import FunctionDefinition


def green_colour_attached_to_name(name: str) -> str:
    return f"{name}, you are now Green :)"


green_colour_attached_to_name_tool: ChatCompletionToolParam = ChatCompletionToolParam(
    type="function",
    function=FunctionDefinition(
        name="green_colour_attached_to_name",
        description="Use this to attach word 'Green' to the provided name.",
        parameters={
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the person",
                },
            },
            "required": ["name"],
            "additionalProperties": False,
        },
        strict=True,
    ),
)

In [2]:
from openai import OpenAI

client = OpenAI()

tools = [green_colour_attached_to_name_tool]

In [5]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Before making a function call, make sure you know all the necessary information from the user.",
        },
        {
            "role": "user",
            "content": "Attach word 'Green' to my name.",
        },
    ],
    tools=tools,
)

In [6]:
for choice in completion.choices:
    print(choice)
    print("---")

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Please provide me with your name so I can attach the word "Green" to it.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))
---


In [189]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Before making a function call, make sure you know all the necessary information from the user.",
        },
        {
            "role": "user",
            "content": "Attach word 'Green' to my name.",
        },
        {
            "role": "assistant",
            "content": "Could you please provide me with your name so I can attach the word 'Green' to it?",
        },
        {"role": "user", "content": "My name is Alex"},
    ],
    tools=tools,
)

In [190]:
completion.choices

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Suxn53cV75hTFrr7mEPAGwed', function=Function(arguments='{"name":"Alex"}', name='green_colour_attached_to_name'), type='function')]))]

In [176]:
from openai.types.chat import ChatCompletionMessageToolCall

if completion.choices[0].message.tool_calls:
    tool_call: ChatCompletionMessageToolCall = completion.choices[0].message.tool_calls[
        0
    ]

In [177]:
tool_call_id = tool_call.id

In [178]:
function_name = tool_call.function.name
arguments_dict = eval(tool_call.function.arguments)

In [179]:
getattr(globals()[function_name], "__call__")(**arguments_dict)

'Alex, you are now Green :)'

In [180]:
result = getattr(globals()[function_name], "__call__")(**arguments_dict)
result

'Alex, you are now Green :)'

In [187]:
from openai.types.chat import (
    ChatCompletionAssistantMessageParam,
    ChatCompletionMessageToolCallParam,
)
from typing import cast


tool_call_message = ChatCompletionAssistantMessageParam(
    role="assistant",
    tool_calls=[
        ChatCompletionMessageToolCallParam(
            **cast(ChatCompletionMessageToolCallParam, tool_call.to_dict())
        )
    ],
)

In [182]:
import json
from openai.types.chat import ChatCompletionToolMessageParam

function_call_result_message = ChatCompletionToolMessageParam(
    role="tool",
    content=json.dumps({"name": "Alex", "function_call_result": result}),
    tool_call_id=tool_call_id,
)

In [183]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Before making a function call, make sure you know all the necessary information from the user.",
        },
        {
            "role": "user",
            "content": "Attach word 'Green' to my name.",
        },
        {
            "role": "assistant",
            "content": "Please provide me with your name so I can attach the word 'Green' to it.",
        },
        {"role": "user", "content": "My name is Alex"},
        tool_call_message,
        function_call_result_message,
    ],
    tools=tools,
)

In [184]:
completion.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Alex, you are now Green! :)', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))